In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from pathlib import Path
from scipy.stats import skew

# En Jupyter, usar el directorio de trabajo actual
BASE_DIR = Path.cwd().parent


def read_data():

    accounts_test = pd.read_csv(f'{BASE_DIR}/data/accounts_test.csv',sep=',')
    accounts_train = pd.read_csv(f'{BASE_DIR}/data/accounts_train_processed.csv',sep=',')
    
    quotes_test = pd.read_csv(f'{BASE_DIR}/data/quotes_test.csv',sep=',')
    quotes_train = pd.read_csv(f'{BASE_DIR}/data/quotes_train.csv',sep=',')

    return accounts_train,accounts_test, quotes_train, accounts_train

accounts_train,accounts_test, quotes_train, accounts_train = read_data()

In [ ]:
# quotes_agg = quotes_train.groupby('account_uuid').agg({
#     'premium': lambda x: x[quotes_train.loc[x.index, 'convert'] == 1].sum(),
#     'product': 'count',  # total de productos cotizados
#     'convert': 'sum'     # total de productos comprados
# })

quotes_agg = quotes_train.groupby("account_uuid").agg(
    num_quotes=("product", "count"),
    num_products_requested=("product", "nunique"),
    sum_premium=("premium", "sum"),
    avg_premium=("premium", "mean"),
    min_premium=("premium", "min"),
    max_premium=("premium", "max"),
    premium_range=('premium',lambda x: x.max() - x.min()),
    carrier_diversity=("carrier_id", lambda x: x.nunique() / len(x)),
    premium_ratio_max_avg=('premium',lambda x: x.max() / (x.mean() + 1e-6)),
    account_value=( 'premium',lambda x: x[quotes_train.loc[x.index, 'convert'] == 1].sum()),
).reset_index()




quotes_agg.reset_index(inplace=True)
#quotes_agg.columns = ['account_uuid','account_value', 'total_quoted_products', 'total_converted_products']

In [35]:
quotes_agg['carrier_diversity'].unique()

array([1.        , 0.5       , 0.66666667, 0.6       , 0.75      ,
       0.33333333, 0.8       , 0.4       , 0.57142857, 0.42857143,
       0.36363636, 0.25      , 0.625     , 0.55555556, 0.375     ,
       0.71428571, 0.83333333, 0.2       , 0.28571429, 0.15384615,
       0.41666667, 0.53846154, 0.18181818, 0.16666667, 0.26666667,
       0.1875    , 0.17241379, 0.3       , 0.29411765, 0.30769231,
       0.14285714])

In [36]:
# ===========================================
# 4. MERGE DE FEATURES
# ===========================================

train_df = accounts_train.merge(quotes_agg, on="account_uuid", how="left")



In [37]:
# ===========================================
# 5. Features based on conversion rates
# ===========================================

# # Tasa de conversión por producto
# product_conv = quotes_train.groupby("product")["convert"].mean().reset_index()
# product_conv.rename(columns={"convert": "product_conversion_rate"}, inplace=True)

# # Tasa de conversión por carrier
# carrier_conv = quotes_train.groupby("carrier_id")["convert"].mean().reset_index()
# carrier_conv.rename(columns={"convert": "carrier_conversion_rate"}, inplace=True)

# # Merging the conversion rates back to the quotes
# quotes_enriched = quotes_train.merge(product_conv, on="product", how="left") \
#                               .merge(carrier_conv, on="carrier_id", how="left")

# # Agregar al nivel cuenta
# conv_agg = quotes_enriched.groupby("account_uuid").agg(
#     avg_product_conversion_rate=("product_conversion_rate", "mean"),
#     avg_carrier_conversion_rate=("carrier_conversion_rate", "mean")
# ).reset_index()



# --- Diversidad de carriers y productos
carrier_stats = quotes_train.groupby("account_uuid")["carrier_id"].agg(
    num_carriers="nunique",
    total_quotes="count",
    carrier_concentration=lambda x: x.value_counts().max() / len(x)
).reset_index()

product_stats = quotes_train.groupby("account_uuid")["product"].agg(
    product_concentration=lambda x: x.value_counts().max() / len(x)
).reset_index()

# --- Dispersión de premiums
premium_stats = quotes_train.groupby("account_uuid")["premium"].agg(
    #std_premium="std",
    iqr_premium=lambda x: x.quantile(0.75) - x.quantile(0.25),
    #premium_skewness="skew"
).reset_index()





# Merge final
# train_df = train_df.merge(conv_agg, on="account_uuid", how="left")
# --- Merge de nuevas features al train_df

train_df = (
    train_df
    .merge(carrier_stats, on="account_uuid", how="left")
    .merge(product_stats, on="account_uuid", how="left")
    .merge(premium_stats, on="account_uuid", how="left")
)

display(train_df.head())
print("Shape final:", train_df.shape)


,account_uuid,state,industry,subindustry,year_established,annual_revenue,total_payroll,business_structure,num_employees,region,...,max_premium,premium_range,carrier_diversity,premium_ratio_max_avg,account_value,num_carriers,total_quotes,carrier_concentration,product_concentration,iqr_premium
0,4de966ca-b13f-4133-acd2-d516328062fc,PA,Retail Trade,Landscaping Services,2012.0,120000.0,50000.0,Limited Liability Company,4.0,Northeast,...,1908.00,731.0,1.0,1.236953,1908.00,2,2,0.5,1.0,365.5
1,85ec3b0c-8f89c-232b-6be4c-d033c5a52e,TX,Contractors,Electrical Contractors and Other Wiring Instal...,2012.0,60000.0,40000.0,Limited Liability Company,1.0,South,...,970.00,0.0,1.0,1.000000,970.00,1,1,1.0,1.0,0.0
2,0fb1f276-95a95-9332-6927f-7e2dc3e610,NY,Unknown,Unknown,2016.0,150000.0,0.0,Corporation,0.0,Northeast,...,816.00,0.0,1.0,1.000000,816.00,1,1,1.0,1.0,0.0
3,5fe23456-f37a8-3c12-7a731-477f60edb9,CA,"Professional, Scientific and Technical Services",Architectural Services,2015.0,75000.0,55000.0,Limited Liability Company,0.0,West,...,1540.00,1040.0,1.0,1.509804,2040.00,2,2,0.5,0.5,520.0
4,2b2ceb23-20eb3-9a3c-87034-2eccac7276,AZ,Retail Trade,Tobacco Stores,2014.0,200000.0,0.0,Limited Liability Company,0.0,West,...,887.52,0.0,1.0,1.000000,887.52,1,1,1.0,1.0,0.0


Shape final: (5709, 26)


In [38]:
# ===========================================
# 6. FEATURES DERIVADAS
# ===========================================

# Premium relativo al revenue
train_df["premium_to_revenue_ratio"] = train_df["avg_premium"] / (train_df["annual_revenue"] + 1)
train_df['log_annual_revenue'] = np.log(train_df['annual_revenue']+ 1)
train_df['log_total_payroll'] = np.log(train_df['total_payroll']+ 1)
train_df['revenue_per_employee'] = train_df['annual_revenue'] / (train_df['num_employees'] + 1)
# Intensidad de cotizaciones por tamaño del negocio
train_df["quotes_per_employee"] = train_df["num_quotes"] / (train_df["num_employees"] + 1)
train_df["quotes_per_million_revenue"] = train_df["num_quotes"] / (train_df["annual_revenue"]/1e6 + 1)

    
train_df['premium_per_employee'] = train_df['sum_premium'] / (train_df['num_employees'] + 1)
train_df['premium_per_revenue'] = train_df['sum_premium'] / (train_df['annual_revenue'] + 1)
train_df['premium_per_quote'] = train_df['sum_premium'] / (train_df['num_quotes'] + 1)


# Interacciones entre features

train_df["max_x_nquotes"] = np.log(train_df["max_premium"] * train_df["num_quotes"] + 1)
train_df["avg_x_nproducts"] = np.log(train_df["avg_premium"] * train_df["num_products_requested"] + 1)
train_df["revenue_x_payroll"] = np.log(train_df["annual_revenue"] * train_df["total_payroll"] + 1)



# train_df['premium_range_bins'] = pd.qcut(train_df['premium_range'], q=10, labels=False, duplicates='drop')


# train_df['premium_log'] = np.log1p(train_df['premium_range'])
# train_df['premium_log_bins'] = pd.qcut(train_df['premium_log'], q=6, labels=False, duplicates='drop')




# Con esto cubro el riesgo de los estados 

# Suma de todas las primas cotizadas en cada estado
state_premium_sum = train_df.groupby('region')['sum_premium'].median()
train_df['state_premium_sum_encoded'] = train_df['region'].map(state_premium_sum)

# Ingreso promedio de empresas en cada estado
state_revenue = train_df.groupby('region')['annual_revenue'].median()
train_df['state_revenue_encoded'] = train_df['region'].map(state_revenue)


# Suma de todas las primas cotizadas en cada estado
business_structure_premium_sum = train_df.groupby('business_structure')['sum_premium'].median()
train_df['business_structure_premium_sum_encoded'] = train_df['business_structure'].map(business_structure_premium_sum)

# Ingreso promedio de empresas en cada estado
business_structure_revenue = train_df.groupby('business_structure')['annual_revenue'].median()
train_df['business_structure_revenue_encoded'] = train_df['business_structure'].map(business_structure_revenue)


# Ingreso promedio de empresas en cada estado
industry_premium_sum = train_df.groupby('industry')['sum_premium'].median()
train_df['industry_sum_premium_encoded'] = train_df['industry'].map(industry_premium_sum)

industry_revenue = train_df.groupby('industry')['annual_revenue'].median()
train_df['industry_revenue_encoded'] = train_df['industry'].map(industry_revenue)

subindustry_premium_sum = train_df.groupby('subindustry')['sum_premium'].median()
train_df['subindustry_sum_premium_encoded'] = train_df['subindustry'].map(subindustry_premium_sum)

subindustry_revenue = train_df.groupby('subindustry')['annual_revenue'].median()
train_df['subindustry_revenue_encoded'] = train_df['subindustry'].map(subindustry_revenue)



display(train_df.head())


,account_uuid,state,industry,subindustry,year_established,annual_revenue,total_payroll,business_structure,num_employees,region,...,premium_log,premium_log_bins,state_premium_sum_encoded,state_revenue_encoded,business_structure_premium_sum_encoded,business_structure_revenue_encoded,industry_sum_premium_encoded,industry_revenue_encoded,subindustry_sum_premium_encoded,subindustry_revenue_encoded
0,4de966ca-b13f-4133-acd2-d516328062fc,PA,Retail Trade,Landscaping Services,2012.0,120000.0,50000.0,Limited Liability Company,4.0,Northeast,...,6.595781,1,1498.0,60000.0,1417.400,70000.0,1503.0,60000.0,1422.19,35000.0
1,85ec3b0c-8f89c-232b-6be4c-d033c5a52e,TX,Contractors,Electrical Contractors and Other Wiring Instal...,2012.0,60000.0,40000.0,Limited Liability Company,1.0,South,...,0.000000,0,1219.0,60000.0,1417.400,70000.0,2009.0,60000.0,2135.00,75000.0
2,0fb1f276-95a95-9332-6927f-7e2dc3e610,NY,Unknown,Unknown,2016.0,150000.0,0.0,Corporation,0.0,Northeast,...,0.000000,0,1498.0,60000.0,1971.075,125000.0,912.0,57500.0,912.00,57500.0
3,5fe23456-f37a8-3c12-7a731-477f60edb9,CA,"Professional, Scientific and Technical Services",Architectural Services,2015.0,75000.0,55000.0,Limited Liability Company,0.0,West,...,6.947937,2,1283.0,60000.0,1417.400,70000.0,1274.0,60000.0,1246.50,80000.0
4,2b2ceb23-20eb3-9a3c-87034-2eccac7276,AZ,Retail Trade,Tobacco Stores,2014.0,200000.0,0.0,Limited Liability Company,0.0,West,...,0.000000,0,1283.0,60000.0,1417.400,70000.0,1503.0,60000.0,962.52,137500.0


In [39]:
# ===========================================
# 7. CHECK FINAL
# ===========================================
print("Shape final:", train_df.shape)
train_df.info()


Shape final: (5709, 48)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5709 entries, 0 to 5708
Data columns (total 48 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   account_uuid                            5709 non-null   object 
 1   state                                   5709 non-null   object 
 2   industry                                5709 non-null   object 
 3   subindustry                             5709 non-null   object 
 4   year_established                        5709 non-null   float64
 5   annual_revenue                          5709 non-null   float64
 6   total_payroll                           5709 non-null   float64
 7   business_structure                      5709 non-null   object 
 8   num_employees                           5709 non-null   float64
 9   region                                  5709 non-null   object 
 10  index                               

In [40]:
train_df.to_csv(f'{BASE_DIR}/data/features.csv', index=False)